In [1]:
!pip install pandas datasets

In [2]:
!pip install transformers torch

In [3]:
!pip install xformers trl peft accelerate bitsandbytes

In [4]:
import json
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

In [5]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("We would be using this device:", device)

We would be using this device: cuda


# Loading Dataset

In [10]:
# Load JSONL data (Custom Dataset)
print("\n2-[1/8] Loading dataset...")
data = []
with open("custom_dataset.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

custom_dataset = Dataset.from_dict({
    "messages": [item["messages"] for item in data]
})

print(f"✓ Loaded {len(custom_dataset)} examples")


2-[1/8] Loading dataset...
✓ Loaded 1004 examples


# Preparing the Train and Test Dataset

we have decided to split 90% for training and 10% for testing

In [11]:
# Split dataset
split_dataset = custom_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]
print(f"✓ Train: {len(train_dataset)} | Val: {len(val_dataset)}")

✓ Train: 903 | Val: 101


In [12]:
print("\n[2/8] Configuring 8-bit quantization...")
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16,
)


[2/8] Configuring 8-bit quantization...


# Model

## Note

we have login inside hugging face and make sure we request for access to use the LLaMA 3.2 3B model

In [13]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [14]:
# Load model and tokenizer
print("\n[3/8] Loading LLaMA 3.2 3B model...")
model_name = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True
)

# LLaMA models usually do not have a pad token by default
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

print("✓ Model loaded in 8-bit")
print("✓ Model size: ~3B parameters")


[3/8] Loading LLaMA 3.2 3B model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✓ Model loaded in 8-bit
✓ Model size: ~3B parameters


In [15]:
# Prepare for k-bit training
print("\n[4/8] Preparing model for QLoRA...")
model = prepare_model_for_kbit_training(
    model,
    use_gradient_checkpointing=True
)

# (Optional but recommended) disable cache during training
model.config.use_cache = False

# Configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


[4/8] Preparing model for QLoRA...


In [16]:
model = get_peft_model(model, lora_config)
print("✓ LoRA adapters added")
model.print_trainable_parameters()

# Tokenization function with proper chat formatting
print("\n[5/8] Preparing tokenization...")

✓ LoRA adapters added
trainable params: 24,313,856 || all params: 3,237,063,680 || trainable%: 0.7511

[5/8] Preparing tokenization...


In [17]:
def format_and_tokenize(example, force=4096):
    # Apply LLaMA chat template
    formatted_text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )

    # Tokenize (NO padding, NO manual labels)
    tokenized = tokenizer(
        formatted_text,
        truncation=True,
        max_length=force,   # IMPORTANT: increase for long transcripts
        padding=False,
    )

    return tokenized


## Note

we need to set the max_length for the tokenization function to 10_000 for the custom dataset

In [18]:
format_and_tokenize_for_cds = lambda x: format_and_tokenize(x, force=10_000)

In [19]:
# Apply tokenization
print("✓ Tokenizing train dataset...")
train_dataset = train_dataset.map(
    format_and_tokenize_for_cds,
    remove_columns=["messages"],
    batched=False,
    desc="Tokenizing train"
)


✓ Tokenizing train dataset...


Tokenizing train:   0%|          | 0/903 [00:00<?, ? examples/s]

In [20]:
print("✓ Tokenizing validation dataset...")
val_dataset = val_dataset.map(
    format_and_tokenize_for_cds,
    remove_columns=["messages"],
    batched=False,
    desc="Tokenizing validation"
)

✓ Tokenizing validation dataset...


Tokenizing validation:   0%|          | 0/101 [00:00<?, ? examples/s]

In [21]:
# Show sample stats (pre-training)
sample = train_dataset[0]

print("\n✓ Sample stats:")
print(f"  - Input length: {len(sample['input_ids'])} tokens")
print(f"  - Attention tokens: {sum(sample['attention_mask'])} tokens")
print(f"  - Truncated: {'Yes' if len(sample['input_ids']) == 2 ** 15 else 'No'}")



✓ Sample stats:
  - Input length: 1382 tokens
  - Attention tokens: 1382 tokens
  - Truncated: No


In [22]:
print("\n[6/8] Configuring training...")
training_args = TrainingArguments(
    output_dir="./llama3.2-3b-qlora-summary",
    num_train_epochs=3,

    # Memory-optimized batch settings
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,

    # Learning settings
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=100,
    max_grad_norm=1.0,

    # Evaluation and saving
    eval_strategy="steps",
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,

    # Optimization
    bf16=True,
    optim="paged_adamw_8bit",
    gradient_checkpointing=True,

    # Efficiency
    group_by_length=True,
    dataloader_num_workers=4,

    # Logging
    logging_steps=10,
    report_to="none",
)


[6/8] Configuring training...


In [23]:
print("✓ Training configuration:")
print(f"  - Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  - Total training steps: ~{len(train_dataset) * 3 // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")
print(f"  - Learning rate: {training_args.learning_rate}")

✓ Training configuration:
  - Effective batch size: 16
  - Total training steps: ~169
  - Learning rate: 0.0002


In [24]:
# Data collator with dynamic padding
print("\n[7/8] Creating data collator...")
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8  # Efficient padding for GPU
)


[7/8] Creating data collator...


In [25]:
# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)


In [26]:
# Start training
print("\n[8/8] Starting training...")
print("=" * 60)
print("TRAINING IN PROGRESS")
print("=" * 60)

trainer.train()


[8/8] Starting training...
TRAINING IN PROGRESS


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
100,2.013100,2.152477


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=171, training_loss=2.1440364199075086, metrics={'train_runtime': 6746.9018, 'train_samples_per_second': 0.402, 'train_steps_per_second': 0.025, 'total_flos': 2.0566782399297946e+17, 'train_loss': 2.1440364199075086, 'epoch': 3.0})

In [27]:
# Save final model
print("\n" + "=" * 60)
print("TRAINING COMPLETE")
print("=" * 60)
print("\nSaving model...")
trainer.save_model("./llama3.2-3b-qlora-summary")
tokenizer.save_pretrained("./llama3.2-3b-qlora-summary")


TRAINING COMPLETE

Saving model...


('./llama3.2-3b-qlora-summary/tokenizer_config.json',
 './llama3.2-3b-qlora-summary/special_tokens_map.json',
 './llama3.2-3b-qlora-summary/chat_template.jinja',
 './llama3.2-3b-qlora-summary/tokenizer.json')

In [28]:
!zip -r ./llama_3b_3_2.zip ./llama3.2-3b-qlora-summary

  adding: llama3.2-3b-qlora-summary/ (stored 0%)
  adding: llama3.2-3b-qlora-summary/README.md (deflated 65%)
  adding: llama3.2-3b-qlora-summary/checkpoint-100/ (stored 0%)
  adding: llama3.2-3b-qlora-summary/checkpoint-100/README.md (deflated 65%)
  adding: llama3.2-3b-qlora-summary/checkpoint-100/adapter_config.json (deflated 58%)
  adding: llama3.2-3b-qlora-summary/checkpoint-100/training_args.bin (deflated 53%)
  adding: llama3.2-3b-qlora-summary/checkpoint-100/special_tokens_map.json (deflated 63%)
  adding: llama3.2-3b-qlora-summary/checkpoint-100/tokenizer_config.json (deflated 96%)
  adding: llama3.2-3b-qlora-summary/checkpoint-100/rng_state.pth (deflated 26%)
  adding: llama3.2-3b-qlora-summary/checkpoint-100/trainer_state.json (deflated 69%)
  adding: llama3.2-3b-qlora-summary/checkpoint-100/chat_template.jinja (deflated 71%)
  adding: llama3.2-3b-qlora-summary/checkpoint-100/optimizer.pt (deflated 11%)
  adding: llama3.2-3b-qlora-summary/checkpoint-100/scheduler.pt (deflate